# Pareto-Optimal Animation

Animated visualization of pareto-optimal aggregation results.

Author: Leander Kotzur

In [ ]:
import os

import numpy as np
import pandas as pd
import plotly.express as px
import tqdm

import tsam
from tsam import ClusterConfig, SegmentConfig
from tsam.timeseriesaggregation import unstackToPeriods

## Load data and configuration

In [ ]:
raw = pd.read_csv("testdata.csv", index_col=0)
raw = raw.rename(
    columns={"T": "Temperature", "Load": "Demand", "Wind": "Wind", "GHI": "Solar"}
)

period_hours = 24
cluster_config = ClusterConfig(method="hierarchical", representation="medoid")

## Load pareto-optimal configurations

In [ ]:
results = pd.read_csv(
    os.path.join("results", "paretoOptimalAggregation.csv"), index_col=0
)
results["time_steps"] = results["segments"] * results["periods"]
results = results[results["time_steps"] > 80]  # Filter very small aggregations

# Add original time series and reverse order (start from full resolution)
results = pd.concat(
    [results, pd.DataFrame([{"segments": 24, "periods": 365}])], ignore_index=True
)
results = results.iloc[::-1].reset_index(drop=True)

In [ ]:
results

## Generate aggregations and build 3D image stack

In [ ]:
def normalize(arr):
    """Normalize array to 0-1 range."""
    return (arr - np.nanmin(arr)) / (np.nanmax(arr) - np.nanmin(arr))


images = []
labels = []

for _, row in tqdm.tqdm(results.iterrows(), total=len(results)):
    segments, periods = int(row["segments"]), int(row["periods"])

    result = tsam.aggregate(
        raw,
        n_periods=periods,
        period_hours=period_hours,
        cluster=cluster_config,
        segments=SegmentConfig(n_segments=segments),
        rescale=False,
    )
    prediction = result.reconstruct()
    reduction = round((1 - segments * periods / len(raw)) * 100, 1)

    # Stack all variables vertically (each normalized to 0-1)
    var_images = []
    for col in raw.columns:
        stacked, _ = unstackToPeriods(prediction[col].copy(), period_hours)
        var_images.append(normalize(stacked.values.T))

    images.append(np.vstack(var_images))
    labels.append(f"{reduction}% reduction ({periods}p x {segments}s)")

# Stack into 3D array: (frames, height, width)
img_stack = np.stack(images)

## Create animated heatmap

In [ ]:
fig = px.imshow(
    img_stack,
    animation_frame=0,
    color_continuous_scale="RdYlBu_r",
    aspect="auto",
    labels={"x": "Day", "y": "Hour", "animation_frame": "Aggregation"},
    title="Time Series Aggregation",
)

# Update slider labels and y-axis ticks
for i, step in enumerate(fig.layout.sliders[0].steps):
    step["label"] = labels[i]

n_hours = period_hours
n_vars = len(raw.columns)
tickvals = [n_hours * i + n_hours // 2 for i in range(n_vars)]
fig.update_yaxes(tickvals=tickvals, ticktext=list(raw.columns))
fig.update_layout(height=600, coloraxis_showscale=False)

fig.show()

## Save as HTML

In [ ]:
fig.write_html(os.path.join("results", "animation.html"))